## 1. PP-msvsr模型简介
PP-MSVSR是一种多阶段视频超分深度架构，具有局部融合模块、辅助损失和细化对齐模块，以逐步细化增强结果。具体来说，在第一阶段设计了局部融合模块，在特征传播之前进行局部特征融合, 以加强特征传播中跨帧特征的融合。在第二阶段中引入了一个辅助损失，使传播模块获得的特征保留了更多与HR空间相关的信息。在第三阶段中引入了一个细化的对齐模块，以充分利用前一阶段传播模块的特征信息。实验证实，PP-MSVSR在Vid4数据集性能优异，仅使用 1.45M 参数PSNR指标即可达到28.13dB！

PP-MSVSR模型由飞桨官方出品，是PaddleGAN自研的视频超分模型。
更多关于PaddleGAN可以点击https://github.com/PaddlePaddle/PaddleGAN 进行了解。

## 2. 模型效果及应用场景
### 2.1 视频超分任务：

#### 2.1.1 数据集：

数据集以REDS数据格式为主，分为训练集和测试集。

#### 2.1.2 模型效果速览：

PP-MSVSR在图片上的超分效果为：

<div align='center'>
<img src='https://user-images.githubusercontent.com/48054808/144848981-00c6ad21-0702-4381-9544-becb227ed9f0.gif' width = "80%"  />
</div>


## 3. 模型如何使用

### 3.1 模型推理：
* 下载 

（不在Jupyter Notebook上运行时需要将"！"或者"%"去掉。）


In [ ]:
%cd ~/work
# 克隆PaddleGAN（从gitee上更快）,本项目已经做持久化处理，不用克隆了。
!git clone https://github.com/PaddlePaddle/PaddleGAN.git

* 安装

In [ ]:
# 运行脚本需在PaddleGAN目录下
%cd ~/work/PaddleGAN/

# 安装所需依赖项【已经做持久化处理，无需再安装】
!pip install -r requirements.txt

# 运行脚本需在PaddleGAN目录下
%cd ~/work/PaddleGAN/
# 设置python运行目录
%env PYTHONPATH=.:$PYTHONPATH
# 设置GPU
%env CUDA_VISIBLE_DEVICES=0  

# 开始安装PaddleGAN 
!python setup.py develop  #如果安装过程中长时间卡住，可中断后继续重新执行，

* 快速体验

恭喜！ 您已经成功安装了PaddleGAN，接下来快速体验视频超分效果

In [ ]:
# 在GPU上预测一段视频
# 视频下载地址：https://user-images.githubusercontent.com/79366697/200290225-7fdd364c-2fbe-48b6-a3bf-87349aedec98.mp4
%cd ~/work/PaddleGAN//applications/
!python tools/video-enhance.py --input demo/Peking_input360p_clip6_5s.mp4 \
                               --process_order PPMSVSR \
                               --output output_dir

会在output_dir文件夹下生成一个超分后的视频。


### 3.2 模型训练：
* 克隆PaddleGAN仓库（详见3.1）

* 准备数据集

这里需要开发者自行准备数据集，以下举例假设开发者已经准备好REDS数据集并且解压到 PaddleGAN/data/REDS/下
通过以下命令确认数据集已经准备完成。
 

In [ ]:
# 查看解压目录
#%cd ~/work/PaddleGAN/
#!tree -d data/REDS

* 修改yaml配置文件



修改配置文件``` configs/msvsr_reds.yaml```

```
total_iters: 150000  # 总的训练迭代次数
output_dir: output_dir  # 模型参数保存路径
find_unused_parameters: True
checkpoints_dir: checkpoints
use_dataset: True
# tensor range for function tensor2img
min_max:
  (0., 1.)

model:
  name: MultiStageVSRModel
  fix_iter: 2500
  generator:
    name: MSVSR
    mid_channels: 32
    num_init_blocks: 2
    num_blocks: 3
    num_reconstruction_blocks: 2
    only_last: True
    use_tiny_spynet: True
    deform_groups: 4
    stage1_groups: 8
    auxiliary_loss: True
    use_refine_align: True
    aux_reconstruction_blocks: 1
    use_local_connnect: True
  pixel_criterion:
    name: CharbonnierLoss
    reduction: mean

dataset:
  train:
    name: RepeatDataset
    times: 1000
    num_workers: 6
    batch_size: 2  #建议使用单机8卡训练，每个卡batch_size为2
    dataset:
      name: VSRREDSMultipleGTDataset
      lq_folder: data/REDS/train_sharp_bicubic/X4  # 训练数据低分辨率图像路径
      gt_folder: data/REDS/train_sharp/X4  # 训练数据高分辨率图像（GT）路径
      ann_file: data/REDS/meta_info_REDS_GT.txt  # 数据集信息文档
      num_frames: 20  # 训练时模型的输入视频帧数
      preprocess:
        - name: GetNeighboringFramesIdx
          interval_list: [1]
        - name: ReadImageSequence
          key: lq
        - name: ReadImageSequence
          key: gt
        - name: Transforms
          input_keys: [lq, gt]
          pipeline:
            - name: SRPairedRandomCrop
              gt_patch_size: 256
              scale: 4
              keys: [image, image]
            - name: PairedRandomHorizontalFlip
              keys: [image, image]
            - name: PairedRandomVerticalFlip
              keys: [image, image]
            - name: PairedRandomTransposeHW
              keys: [image, image]
            - name: TransposeSequence
              keys: [image, image]
            - name: NormalizeSequence
              mean: [0., 0., 0.]
              std: [255., 255., 255.]
              keys: [image, image]

  test:
    name: VSRREDSMultipleGTDataset
    lq_folder: data/REDS/REDS4_test_sharp_bicubic/X4  # 测试/验证数据低分辨率图像路径
    gt_folder: data/REDS/REDS4_test_sharp/X4  # 测试/验证数据高分辨率图像（GT）路径
    ann_file: data/REDS/meta_info_REDS_GT.txt  # 数据集信息文档
    num_frames: 100  # 测试/验证时模型的输入视频帧数
    test_mode: True
    preprocess:
        - name: GetNeighboringFramesIdx
          interval_list: [1]
        - name: ReadImageSequence
          key: lq
        - name: ReadImageSequence
          key: gt
        - name: Transforms
          input_keys: [lq, gt]
          pipeline:
            - name: TransposeSequence
              keys: [image, image]
            - name: NormalizeSequence
              mean: [0., 0., 0.]
              std: [255., 255., 255.]
              keys: [image, image]

lr_scheduler:
  name: CosineAnnealingRestartLR
  learning_rate: !!float 2e-4  # 学习率
  periods: [150000]
  restart_weights: [1]
  eta_min: !!float 1e-7

optimizer:
  name: Adam
  # add parameters of net_name to optim
  # name should in self.nets
  net_names:
    - generator
  beta1: 0.9
  beta2: 0.99

validate:
  interval: 5000  # 多少个迭代次进行验证
  save_img: false  # 验证时是否保存图像

  metrics:
    psnr: # metric name, can be arbitrary
      name: PSNR  # 验证指标 PSNR
      crop_border: 0
      test_y_channel: false
    ssim:
      name: SSIM  # 验证指标 SSIM
      crop_border: 0
      test_y_channel: false

log_config:
  interval: 10  # 每多少个迭代次显示
  visiual_interval: 5000  # 多少个迭代次可视化

snapshot_config:
  interval: 5000  # 多少个迭代次保存一次

export_model:
  - {name: 'generator', inputs_num: 1}

```


* 训练模型

In [ ]:
%cd ~/work/PaddleGAN/
%env CUDA_VISIBLE_DEVICES=0
#开始训练
!python -u tools/main.py --config-file configs/msvsr_reds.yaml

* 模型评估

模型评估也是使用配置文件```configs/msvsr_reds.yaml```以REDS数据集为例，需要下载好REDS数据集的验证数据集并且解压到 PaddleGAN/data/REDS/下，在配置文件中更改验证数据集路径后，使用如下命令进行评估。

In [ ]:
%cd ~/work/PaddleGAN/
%env CUDA_VISIBLE_DEVICES=0
#训练完以后，进行评估
!python tools/main.py --config-file configs/msvsr_reds.yaml --evaluate-only --load ${PATH_OF_WEIGHT}

## 4. 模型原理


* 与单幅图像超分（SISR）任务不同，视频超分（VSR）任务的关键是充分利用跨帧之间的互补信息来重建高分辨率图像序列。由于来自不同帧的图像具有不同的运动和场景，准确的对齐多个帧和有效的融合不同帧一直是 VSR 任务的研究重点。为了利用相邻帧的丰富互补信息，PP-MSVSR设计了一种多阶段 VSR 深度架构，包括局部融合模块、辅助损失和再对齐模块来逐步增强超分效果。
* PP-MSVSR结合了滑动窗口方法和循环网络方法的思想，使用多阶段策略进行视频超分，设计了局部融合模块、辅助损失和细化对齐模块以逐步细化增强结果。
<div align="center">
<img src="https://user-images.githubusercontent.com/79366697/200302378-cd0a5f1b-5d89-44a5-aae3-8f4c587a3575.png" width = "80%"  />
</div>

* 受滑动窗口方法思想的启发，PP-MSVSR在第一阶段设计了一个局部融合模块LFM。该模块在特征传播之前先进行局部特征融合，以加强特征传播中的跨帧特征融合。 具体来说，LFM的目的是让当前帧的特征先融合其相邻帧的信息，然后将融合后的特传给下一阶段的传播模块。
<div align="center">
<img src="https://user-images.githubusercontent.com/79366697/200302612-6ac31db1-ac32-4b14-af3e-479f66746ac9.png" width = "80%"  />
</div>

* 受到循环网络的启发，PP-MSVSR在第二阶段使用与 basicvsr++ 相同的双向循环结构来融合和传播特征，并设计了一个辅助损失，使特征更接近真实高分辨率特征空间。 具体来说，对第二阶段传播后的特征上采样后添加辅助损失。
* 与图像超分不同，视频超分通常需要将相邻帧与当前帧对齐以更好地整合相邻帧的信息。 在一些大型运动视频超分任务中，对齐的作用尤为明显。在使用双向循环网络的过程中，往往会有多次相同的对齐操作。为了充分利用之前对齐操作的结果，PP-MSVSR提出了细化对齐模块RAM，可以利用之前对齐的参数并获得更好的对齐结果。
<div align="center">
<img src="https://user-images.githubusercontent.com/79366697/200302619-87b10406-580b-4a65-a6bd-bbb1525005b1.png" width = "80%"  />
</div>


## 5. 相关论文以及引用信息
```
@article{jiang2021PP-MSVSR,
  author = {Jiang, Lielin and Wang, Na and Dang, Qingqing and Liu, Rui and Lai, Baohua},
  title = {PP-MSVSR: Multi-Stage Video Super-Resolution},
  booktitle = {arXiv preprint arXiv:2112.02828},
  year = {2021}
  }
```
